In [1]:
import cv2
import mediapipe as mp

In [11]:
from pynput.mouse import Button, Controller
import pyautogui

In [30]:
import time


In [12]:
mouse=Controller()

screen width,height

In [15]:
screenwidth,screenheight=pyautogui.size()

function index finger value  (tip)

In [17]:
def index_finger_tip(landmarks):
    index_tip=(int(landmarks.landmark[8].x*screenwidth),int(landmarks.landmark[8].y*screenheight))
    return index_tip



In [4]:
mp_hands=mp.solutions.hands
mp_drawing_utils=mp.solutions.drawing_utils
hand=mp_hands.Hands(max_num_hands=1)

In [2]:
def count(landmarks):
    tip=[8,12,16,20]
    pip=[7,11,15,19]
    c=0
    for i,j in zip(tip,pip):
        if landmarks.landmark[i].y<landmarks.landmark[j].y:
            c+=1
    if landmarks.landmark[3].x>landmarks.landmark[17].x:
        if landmarks.landmark[4].x>landmarks.landmark[2].x:
            c+=1
    elif landmarks.landmark[3].x<landmarks.landmark[17].x:
        if landmarks.landmark[4].x<landmarks.landmark[2].x:
            c+=1
    return c

In [52]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if ret==False:
        break
    h,w,_=frame.shape
    frame=cv2.flip(frame,1)
    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results=hand.process(rgb_img).multi_hand_landmarks
    if results:
        landmarks=results[0]
        dectect_gesture(landmarks)
        mp_drawing_utils.draw_landmarks(frame,landmarks,mp_hands.HAND_CONNECTIONS)
    cv2.imshow("mouse",frame)
    if cv2.waitKey(10)==ord('q'):
        cv2.destroyAllWindows()
        cap.release()

dectect gesture


In [51]:
def dectect_gesture(landmarks):
    index_tip=index_finger_tip(landmarks)
    fingers=count(landmarks)
    #move cursor
    if fingers==1 and landmarks.landmark[8].y<landmarks.landmark[6].y:
        pyautogui.moveTo(index_tip)
    #left click
    elif fingers==2 and (landmarks.landmark[8].y<landmarks.landmark[6].y) and (landmarks.landmark[12].y<landmarks.landmark[10].y):
        mouse.click(Button.left)
        mouse.release(Button.left)
        time.sleep(0.4)
        #right click
    elif fingers==3 and landmarks.landmark[8].y<landmarks.landmark[6].y:
        mouse.click(Button.right)
        mouse.release(Button.right)
        time.sleep(0.4)
    #double click
    elif fingers==0 :
        mouse.click(Button.left,2)
        mouse.release(Button.left)
        time.sleep(0.4)
    #screen shot
    elif fingers==5:
        screenshot=pyautogui.screenshot()
        screenshot.save(f"img{landmarks.landmark[8].x*screenwidth}.jpg")
        time.sleep(1)